In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from transformers import pipeline
from torch.utils.data import Dataset, DataLoader
import os
import huggingface_hub
huggingface_hub.login("xxx")

In [16]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

c:\anaconda3\envs\study64\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\양진우\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [19]:
model.eval()

prompt = "Which is a fruit? (a) apple, (b) car, (c) dog"
inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_k=10
        )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Which is a fruit? (a) apple, (b) car, (c) dog, (d) apple
A. b
B. d
C. c
D. a
Answer: D


In [23]:
# 테스트 데이터셋 정의
test_questions = [
    "Which is a furniture? (a) chair, (b) dog, (c) pencil",
    "Which is a planet? (a) Mars, (b) Moon, (c) Sun",
    "Which is a color? (a) happy, (b) blue, (c) fast",
    "Which is a vehicle? (a) banana, (b) car, (c) tree",
    "Which is an animal? (a) house, (b) phone, (c) lion",
    "Which is a vegetable? (a) carrot, (b) book, (c) chair",
    "Which is a drink? (a) table, (b) water, (c) shoe",
    "Which is a sport? (a) sleep, (b) eat, (c) tennis",
    "Which is a weather? (a) rainy, (b) smile, (c) walk",
    "Which is a job? (a) doctor, (b) cloud, (c) song"
]

test_answers = ['a', 'a', 'b', 'b', 'c', 'a', 'b', 'c', 'a', 'a']

In [24]:
# 예측 실행
model.eval()
predictions = []

for question in test_questions:
    inputs = tokenizer(question, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_k=10
        )
    
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(prediction)
    print(f"질문: {question}")
    print(f"모델 답변: {prediction}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a furniture? (a) chair, (b) dog, (c) pencil
모델 답변: Which is a furniture? (a) chair, (b) dog, (c) pencil, (d) bed.
Which is not a furniture? (a) table, (b) door, (c) lamp, (d) bookcase.
What is a furniture? (a) chair, (b) dog, (c) pencil, (d) bed.
What is not a furniture? (a) table, (b) door, (c) lamp, (d) bookcase.
Which is a furniture? (a) chair, (b) dog, (c) pencil, (d) bed.
Which is not a furniture? (a) table, (b) door, (c) lamp, (d) bookcase.
What is a furniture? (a) chair, (b) dog, (c) pencil, (d) bed.
What is not a furniture? (a) table, (b) door, (c) lamp, (d) bookcase.
What is a furniture? (a) chair, (



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a planet? (a) Mars, (b) Moon, (c) Sun
모델 답변: Which is a planet? (a) Mars, (b) Moon, (c) Sun, (d) Jupiter.
Which is a satellite? (a) Moon, (b) Mars, (c) Venus, (d) Earth.
Which is a moon? (a) Moon, (b) Earth, (c) Venus, (d) Jupiter.
Which is a planet? (a) Earth, (b) Sun, (c) Moon, (d) Mars.
Which is a satellite? (a) Sun, (b) Mars, (c) Moon, (d) Venus.
Which is a moon? (a) Earth, (b) Mars, (c) Sun, (d) Moon.
Which is a planet? (a) Moon, (b) Sun, (c) Earth, (d) Venus.
Which is a satellite? (a) Earth, (b) Sun, (c) Moon, (d) Mars.
Which is a moon? (a) Sun, (b) Moon, (c) Venus



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

질문: Which is a drink? (a) table, (b) water, (c) shoe
모델 답변: Which is a drink? (a) table, (b) water, (c) shoe, (d) cup, (e) bottle
Answer: d
Explanation: A drink is a liquid that is usually consumed with food. A cup of water is a drink, but a shoe is not.



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a sport? (a) sleep, (b) eat, (c) tennis
모델 답변: Which is a sport? (a) sleep, (b) eat, (c) tennis, (d) walk
Which is a sport? (a) sleep, (b) eat, (c) tennis, (d) walk
The answer is (a) sleep.



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a weather? (a) rainy, (b) smile, (c) walk
모델 답변: Which is a weather? (a) rainy, (b) smile, (c) walk, (d) snow,copy and paste the word in the box. Which is a weather? (a) rainy, (b) smile, (c) walk, (d) snow.
Which is a weather? (a) rainy, (b) smile, (c) walk, (d) snow,copy and paste the word in the box. Which is a weather? (a) rainy, (b) smile, (c) walk, 형태소 분석서 (d) snow.
Which is a weather? (a) rainy, (b) smile, (c) walk, (d) snow,copy and paste the word in the box. Which is a weather? (a) rainy, (b) smile, (c) walk, (d) snow.
Which is a weather? (a) rainy, (b) smile, (c) walk, (d) snow,copy and paste the word in the box. Which is a weather? (a

질문: Which is a job? (a) doctor, (b) cloud, (c) song
모델 답변: Which is a job? (a) doctor, (b) cloud, (c) song, (d) dog.
A. a
B. b
C. c
D. d
Answer: A

